In [1]:
import os
# import tensorflow.compat.v2 as tf
import tensorflow as tf
import json
from tensorflow.keras import datasets, layers, models, metrics
from minio import Minio
from minio.error import ResponseError

In [9]:
!rm -f /tmp/mnist.npz
!mc rm myminio/dataset/mnist/mnist.npz

!wget https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
!mc mb myminio/model/mnist-model
!mc cp mnist.npz myminio/dataset/mnist/

Removing `myminio/dataset/mnist/mnist.npz`.
--2020-12-03 07:33:25--  https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 74.125.203.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11490434 (11M) [application/octet-stream]
Saving to: ‘mnist.npz.8’

mnist.npz.8         100%[===================>]  10.96M  40.2MB/s    in 0.3s    

2020-12-03 07:33:26 (40.2 MB/s) - ‘mnist.npz.8’ saved [11490434/11490434]

Bucket created successfully `myminio/model/mnist-model`.
mnist.npz:     52.87 MiB / 52.87 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 110.95 MiB/s 0s

In [3]:
minioClient = Minio('minio-service.kubeflow:9000',
              access_key='minio', secret_key='minio123', secure=False)
try:
    data = minioClient.get_object('dataset', 'mnist/mnist.npz')
    with open('/tmp/mnist.npz', 'wb') as file_data:
        for d in data.stream(32*1024):
            file_data.write(d)
except ResponseError as err:
    print(err)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='/tmp/mnist.npz')
x_train, x_test = x_train / 255.0, x_test / 255.0

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 731c5ac602752760c8e48fbffcf8c3b850d9dc2a2aedcf2cc48468fc17b673d1 so we will re-download the data.
11493376/11490434 [==============================] - 0s 0us/step


In [4]:
dropout_rate = 0.3
learning_rate = 0.1

In [7]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

sgd = tf.keras.optimizers.SGD(lr=learning_rate,
                                  decay=1e-6,
                                  momentum=0.9,
                                  nesterov=True)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3)
score = model.evaluate(x_test,  y_test, verbose=2)
loss = score[0]
accuracy = score[1]

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 5s 89us/sample - loss: 0.4503 - accuracy: 0.8697
Epoch 2/3
60000/60000 [==============================] - 5s 88us/sample - loss: 0.3518 - accuracy: 0.9052
Epoch 3/3
60000/60000 [==============================] - 5s 76us/sample - loss: 0.3134 - accuracy: 0.9178
10000/10000 - 1s - loss: 0.1975 - accuracy: 0.9510


In [ ]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',	# Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0' 	# If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})    
model.save("s3://model/mnist-model")

In [ ]:
print(loss)
print(accuracy)